In [1]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
import os

load_dotenv()
apikey = os.environ.get("apikey")
snp500 = pd.DataFrame(fmpsdk.sp500_constituent(apikey))

PBR_KEY = 'priceBookValueRatio'

from fetch import Fetcher

file_directories = ['financial-ratios-quarter-20220820.csv']
key_list = [PBR_KEY]
key = key_list[0]

fetcher = Fetcher()
fetcher.register(key_list=key_list, csv_file_directory=file_directories[0])
from strategy import *

selector_pbr = Selector(type=SelectorType.HORIZON_RANGE, key=PBR_KEY, value=(0.0, 2.0), horizon=4)
selector_sequence = SelectorSequence(selectors=[selector_pbr])
filter_1 = Filter(selector_sequence_union=[selector_sequence])


In [2]:
initial_holdings = [Asset(symbol=symbol) for symbol in snp500['symbol']]
start_date = '2017-03-01'
end_date = '2019-03-01'
strategist = Strategist(fetcher=fetcher, asset_pool=initial_holdings, start_date=start_date, end_date=end_date)

strategist.select_assets(filter=filter_1, rebalance_date='2018-03-06')
strategist.select_assets(filter=filter_1, rebalance_date='2018-09-06')


In [3]:
from analyze import Analyst
analyst = Analyst()
analyst.register(table_name='price_list', csv_file_directory='snp500_price_list_19620102_20220814.csv')
analyst.register(table_name='economic_indicators', csv_file_directory='snp500_price_list_19620102_20220814.csv')

In [4]:
total_initial = 1
total_history = pd.Series(dtype=float)

for segment in strategist.state:
    segment_start_date = segment.start_date
    segment_end_date = segment.end_date
    symbols = segment.get_asset_symbol_list()
    table_key = 'daily'
    df_price_table = analyst.table_dict['price_list'][table_key]
    df_segment_price_history = df_price_table[segment_start_date:segment_end_date][symbols]



In [18]:
df_segment_price_history_copy = df_segment_price_history.copy()
initial_price_list = df_segment_price_history.iloc[0]
holding_list = initial_price_list.apply(lambda x: 1.0/x/len(initial_price_list))

sum(holding_list * initial_price_list)

0.9999999999999987

In [25]:
df_segment_price_history_copy.apply(lambda x: x.dot(holding_list),axis=1)

date
2018-09-06    1.000000
2018-09-07    0.994095
2018-09-10    0.994601
2018-09-11    0.995366
2018-09-12    0.994474
                ...   
2019-02-25    0.976076
2019-02-26    0.972945
2019-02-27    0.972855
2019-02-28    0.972005
2019-03-01    0.977004
Length: 121, dtype: float64

In [26]:
df_segment_price_history_copy

,USB,SJM,BRK-B,EVRG,DD,C,CB,KMI,LNC,GS,...,ALL,KHC,LUMN,ZBH,HIG,PLD,MOS,RE,SBNY,LEN
date,,,,,,,,,,,,,,,,,,,,,
2018-09-06,47.813110,97.996849,212.88,51.879562,92.147766,62.018902,127.530273,14.108843,57.818901,217.002487,...,93.343987,47.369213,15.910275,116.401573,45.746872,61.330840,29.321867,202.170670,112.390556,50.636799
2018-09-07,47.689911,97.782593,214.04,51.307526,92.042564,61.603130,126.852425,14.228613,57.562325,216.438080,...,92.356071,47.194088,15.866886,116.608818,45.480896,60.253579,28.992620,198.649231,112.192467,48.885693
2018-09-10,47.637108,98.353950,215.11,51.465939,92.121445,61.594276,125.784584,14.124809,57.394226,214.587448,...,90.343323,47.736172,15.772872,116.052994,45.288296,60.746563,29.341228,193.440781,112.117004,49.040489
2018-09-11,47.698700,97.479065,215.10,51.404335,92.147766,61.417366,125.877434,14.380320,57.208420,213.014450,...,90.629547,48.169823,16.279106,116.128334,45.196579,60.609623,29.951296,194.058441,112.598068,49.504871
2018-09-12,47.126675,98.380730,214.59,51.175514,91.043243,62.372723,125.041695,14.500089,56.792591,211.108292,...,90.094040,49.304016,16.590082,118.879173,44.893929,60.646140,30.135284,197.284897,110.428558,50.520702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-25,46.108753,91.650147,202.31,49.357437,74.232224,57.879814,127.091843,15.774675,56.704483,184.556198,...,88.074043,28.889637,9.813415,117.213417,45.403305,66.031292,31.778099,205.756668,129.260895,48.113373
2019-02-26,45.831791,96.211807,201.90,49.490509,73.967468,57.512077,126.904068,15.709287,56.561260,184.788437,...,87.322296,27.952814,9.671840,116.702583,45.255558,65.827435,30.488354,205.397171,128.889511,47.938908
2019-02-27,46.046204,97.554008,202.29,49.224384,72.927948,57.763222,124.894943,15.758325,56.400143,184.788437,...,87.686165,27.176348,9.761257,117.771591,45.320202,65.012005,31.196257,206.761520,129.927414,47.337978
